In [62]:
import xarray as xr
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import netCDF4 as nc
import os
import zipfile
#%pip install tqdm

In [63]:
def saveNC(fn,Resultados,fechaI='2016-01-01'):
    n=Resultados.shape[1]
    
    ds=nc.Dataset(fn,'w',format='NETCDF4')

    lon = ds.createDimension('lon', 133)
    lat = ds.createDimension('lat', 198)
    time = ds.createDimension('time', n)
    times = ds.createVariable('time', 'f4', ('time',))
    lons = ds.createVariable('lon', 'f4', ('lon',))
    lats = ds.createVariable('lat', 'f4', ('lat',))
    value = ds.createVariable('value', 'f4', ('time', 'lat', 'lon',))
    value.units = 'Unknown'
    times=pd.date_range(fechaI, periods=n, freq='d')
    lats[:] = np.linspace(start=0.95, stop=-18.75,num=198)
    lons[:] = np.linspace(start=-81.25, stop=-68.05,num=133)
    for i in range(n):
        value[i,:,:]=Resultados[:,i].reshape(198,133)
        
    ds.close()
    
def matrixinfo(archivo,tipo='PISCO'):
    ds = xr.open_dataset(archivo)
    ds=ds.to_array()
    
    if tipo=='PISCO':
        ds=ds[1,0:432,:,:]
    else:
        ds=ds[1,:,:,:]
    ds=np.array(ds)
    
    n=ds.shape[0]
    if n==432:
        ds=ds.reshape(432,26334).transpose()
    else:
        print(n)
        ds=ds.reshape(n,26334).transpose()
    return ds

In [64]:
#http://blog.yhat.com/tutorials/rpy2-combing-the-power-of-r-and-python.html

import rpy2.robjects.packages as rpackages
import rpy2.robjects as robjects
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
import datetime as dt
utils = rpackages.importr('utils')
base = rpackages.importr('base')
utils.chooseCRANmirror(ind=1)
hyfo=rpackages.importr('hyfo')
raster = rpackages.importr('raster')
qmap=rpackages.importr('qmap')
dplyr=rpackages.importr('dplyr')

In [65]:
times=pd.date_range('1981-01-01', periods=9131, freq='d')

In [66]:
rstring="""

     function(PISCO,GCM_data,FUT,date,n){
    library(qmap)
    library(lubridate)
    wet_day=TRUE
    
    Date = seq(as.Date("1981-01-01"), length.out = 432, by = "m")
    his_obs <- data.frame(Date = Date,
                           value = as.numeric(PISCO))
    his_mod <- data.frame(Date = Date,
                           value = GCM_data)
                           
                           
    ### Quitando años niño 1983 y 1998
    his_obs <- his_obs[which(!year(his_obs$Date) %in% c(1983,1998)),]
    
    his_mod <- his_mod[which(!year(his_mod$Date) %in% c(1983,1998)),]

      ########################################33
    Date = seq(as.Date(date), length.out = n, by = "m")
    
    fut_mod <- data.frame(Date = Date,
                           value = as.numeric(FUT))
    

    eqm_qmap_s <- function(his_mod,
                       his_obs,
                       fut_mod,
                       wet_day)
    {
  
      do.call(rbind,
              lapply(list(c("12", "01", "02"),
                          c("03", "04", "05"),
                          c("06", "07", "08"),
                          c("09", "10", "11")
                          ), 
                     function(x){

                       his_obs_m <- his_obs[format(his_obs$Date, "%m") %in% x, ]
                       his_mod_m <- his_mod[format(his_mod$Date, "%m") %in% x, ]
                       fut_mod_m <- fut_mod[format(fut_mod$Date, "%m") %in% x, ]

                       eqm_model <- qmap::fitQmapQUANT(his_obs_m$value,
                                                       his_mod_m$value,
                                                       wet.day = wet_day,
                                                       qstep = 0.01)

                       fut_mod_m_c <- qmap::doQmapQUANT(fut_mod_m$value, 
                                                        eqm_model, 
                                                        type = "linear")

                       response <- fut_mod_m
                       response[, colnames(fut_mod_m)[1]] = fut_mod_m[,1]
                       response[, colnames(fut_mod_m)[2]] = round(fut_mod_m_c, 1)
                       response
                     }
              )
      ) -> fut_mod_c

      fut_mod_c[order(fut_mod_c$Date),]
  
    }
    eqm_qmap_s(his_mod = his_mod,
                         his_obs = his_obs,
                         fut_mod = fut_mod,
                         wet_day = wet_day)
}
"""

rfunc=robjects.r(rstring)

In [77]:
PISCO=matrixinfo("/home/jose/Documents/PISCO/PISCOm.nc",tipo='PISCO')
Elements=[]
for n in range(PISCO.shape[0]):
    if np.sum(np.isnan(PISCO[n,:]))==0:
        Elements.append(n)


In [78]:
future_paths=['/home/jose/Documents/GCM/GCM_RCP45_corregido/PP/Escenarios_Horizontes_resampleado/future/'+x.split('.')[0]+'_F.nc' for x in os.listdir('/home/jose/Documents/GCM/GCM_RCP45_corregido/PP/Escenarios_Horizontes_resampleado/present/')]

In [79]:
present_paths=['/home/jose/Documents/GCM/GCM_RCP45_corregido/PP/Escenarios_Horizontes_resampleado/present/'+x for x in os.listdir('/home/jose/Documents/GCM/GCM_RCP45_corregido/PP/Escenarios_Horizontes_resampleado/present/')]

# Save path futures


In [80]:
save_paths=['/home/jose/Documents/GCM/GCM_RCP45_corregido/PP/Escenarios_Horizontes_corregidos/'+x.split('.')[0]+'_F_C.nc' for x in os.listdir('/home/jose/Documents/GCM/GCM_RCP45_corregido/PP/Escenarios_Horizontes_resampleado/present/')]

# Save path present

In [15]:
save_paths=['/home/jose/Documents/GCM/GCM_RCP45_corregido/PP/Escenarios_Horizontes_corregidos/'+x.split('.')[0]+'_P_C.nc' for x in os.listdir('/home/jose/Documents/GCM/GCM_RCP45_corregido/PP/Escenarios_Horizontes_resampleado/present/')]

In [81]:
FECHAS=['2017-01-1']


for i in range(len(present_paths)):

    GCM=matrixinfo(present_paths[i],tipo='GCM')  
    FCR=matrixinfo(future_paths[i],'ff')
    #FCR=matrixinfo(present_paths[i],tipo='GCM') #Para corregir los presentes
    file_saved=save_paths[i]
    nn=FCR.shape[1]
    Resultados=np.zeros((PISCO.shape[0],nn))
    Resultados[:] = np.NAN
    for pixel in tqdm(Elements):
        hist_obs=base.as_numeric(PISCO[pixel,:])
        hist_mod=base.as_numeric(GCM[pixel,:])
        fut_mod=base.as_numeric(FCR[pixel,:])
        try:
            Resultados[pixel,:]=rfunc(hist_obs,hist_mod,fut_mod,FECHAS[0],nn).value
            if pixel%100==0:
                print("Iteracion el el pixel {}".format(pixel))
        except:
            print("An exception occurred")

        #Resultados[Resultados<0]=np.NAN

        #saveNC(fn=file_saved,Resultados=Resultados,fechaI=FECHAS[0])
    saveNC(fn=file_saved,Resultados=Resultados,fechaI=FECHAS[0])      

  0%|          | 17/19617 [00:00<01:57, 167.20it/s]

1008


  1%|          | 115/19617 [00:00<01:59, 162.90it/s]

Iteracion el el pixel 100


  1%|          | 200/19617 [00:01<01:59, 162.34it/s]

...

Iteracion el el pixel 25400


100%|█████████▉| 19540/19617 [02:01<00:00, 130.61it/s]

Iteracion el el pixel 25800


100%|██████████| 19617/19617 [02:01<00:00, 161.06it/s]


Iteracion el el pixel 26200


# TMAX

In [24]:
rstring="""

     function(PISCO,GCM_data,FUT,date,n){
    library(qmap)
    library(lubridate)
    wet_day=FALSE
    
    Date = seq(as.Date("1981-01-01"), length.out = 432, by = "m")
    his_obs <- data.frame(Date = Date,
                           value = as.numeric(PISCO))
    his_mod <- data.frame(Date = Date,
                           value = GCM_data)
                           
                           
    ### Quitando años niño 1983 y 1998
    his_obs <- his_obs[which(!year(his_obs$Date) %in% c(1983,1998)),]
    
    his_mod <- his_mod[which(!year(his_mod$Date) %in% c(1983,1998)),]

      ########################################33
    Date = seq(as.Date(date), length.out = n, by = "m")
    
    fut_mod <- data.frame(Date = Date,
                           value = as.numeric(FUT))
    

    eqm_qmap_s <- function(his_mod,
                       his_obs,
                       fut_mod,
                       wet_day)
    {
  
      do.call(rbind,
              lapply(list(c("12", "01", "02"),
                          c("03", "04", "05"),
                          c("06", "07", "08"),
                          c("09", "10", "11")
                          ), 
                     function(x){

                       his_obs_m <- his_obs[format(his_obs$Date, "%m") %in% x, ]
                       his_mod_m <- his_mod[format(his_mod$Date, "%m") %in% x, ]
                       fut_mod_m <- fut_mod[format(fut_mod$Date, "%m") %in% x, ]

                       eqm_model <- qmap::fitQmapQUANT(his_obs_m$value,
                                                       his_mod_m$value,
                                                       wet.day = wet_day,
                                                       qstep = 0.01)

                       fut_mod_m_c <- qmap::doQmapQUANT(fut_mod_m$value, 
                                                        eqm_model, 
                                                        type = "linear")

                       response <- fut_mod_m
                       response[, colnames(fut_mod_m)[1]] = fut_mod_m[,1]
                       response[, colnames(fut_mod_m)[2]] = round(fut_mod_m_c, 1)
                       response
                     }
              )
      ) -> fut_mod_c

      fut_mod_c[order(fut_mod_c$Date),]
  
    }
    eqm_qmap_s(his_mod = his_mod,
                         his_obs = his_obs,
                         fut_mod = fut_mod,
                         wet_day = wet_day)
}
"""

rfunc=robjects.r(rstring)

In [25]:
PISCO=matrixinfo("/home/jose/Documents/PISCO/PISCO_max.nc",tipo='PISCO')
Elements=[]
for n in range(PISCO.shape[0]):
    if np.sum(np.isnan(PISCO[n,:]))==0:
        Elements.append(n)

<b> rutas de archivos y de guardado

In [36]:
future_paths=['/home/jose/Documents/GCM/GCM_RCP45_corregido/TMAX/Escenarios_Horizontes_resampleado/future/'+x.split('.')[0]+'_F.nc' for x in os.listdir('/home/jose/Documents/GCM/GCM_RCP45_corregido/TMAX/Escenarios_Horizontes_resampleado/present/')]

In [37]:
present_paths=['/home/jose/Documents/GCM/GCM_RCP45_corregido/TMAX/Escenarios_Horizontes_resampleado/present/'+x for x in os.listdir('/home/jose/Documents/GCM/GCM_RCP45_corregido/TMAX/Escenarios_Horizontes_resampleado/present/')]

In [38]:
save_paths=['/home/jose/Documents/GCM/GCM_RCP45_corregido/TMAX/Escenarios_Horizontes_corregidos/'+x.split('.')[0]+'_F_C.nc' for x in os.listdir('/home/jose/Documents/GCM/GCM_RCP45_corregido/TMAX/Escenarios_Horizontes_resampleado/present/')]

In [31]:
save_paths=['/home/jose/Documents/GCM/GCM_RCP45_corregido/TMAX/Escenarios_Horizontes_corregidos/'+x.split('.')[0]+'_P_C.nc' for x in os.listdir('/home/jose/Documents/GCM/GCM_RCP45_corregido/TMAX/Escenarios_Horizontes_resampleado/present/')]

In [39]:
FECHAS=['2017-01-1']


for i in range(len(present_paths)):

    GCM=matrixinfo(present_paths[i],tipo='GCM')  
    FCR=matrixinfo(future_paths[i],'ff')
    #FCR=matrixinfo(present_paths[i],'ff')
    file_saved=save_paths[i]
    nn=FCR.shape[1]
    Resultados=np.zeros((PISCO.shape[0],nn))
    Resultados[:] = np.NAN
    for pixel in tqdm(Elements):
        hist_obs=base.as_numeric(PISCO[pixel,:])
        hist_mod=base.as_numeric(GCM[pixel,:])
        fut_mod=base.as_numeric(FCR[pixel,:])
        try:
            Resultados[pixel,:]=rfunc(hist_obs,hist_mod,fut_mod,FECHAS[0],nn).value
            if pixel%100==0:
                print("Iteracion el el pixel {}".format(pixel))
        except:
            print("An exception occurred")

        #Resultados[Resultados<0]=np.NAN

        #saveNC(fn=file_saved,Resultados=Resultados,fechaI=FECHAS[0])
    saveNC(fn=file_saved,Resultados=Resultados,fechaI=FECHAS[0])   

  0%|          | 16/19791 [00:00<02:09, 152.39it/s]

1008


  1%|          | 115/19791 [00:00<02:03, 159.86it/s]

Iteracion el el pixel 100


  1%|          | 197/19791 [00:01<02:02, 159.69it/s]

...

Iteracion el el pixel 25400


100%|█████████▉| 19719/19791 [02:04<00:00, 159.47it/s]

Iteracion el el pixel 25800


100%|██████████| 19791/19791 [02:04<00:00, 158.84it/s]

Iteracion el el pixel 26200


# Tmin

In [47]:
future_paths=['/home/jose/Documents/GCM/GCM_RCP45_corregido/TMIN/Escenarios_Horizontes_resampleado/future/'+x.split('.')[0]+'_F.nc' for x in os.listdir('/home/jose/Documents/GCM/GCM_RCP45_corregido/TMIN/Escenarios_Horizontes_resampleado/present/')]

In [48]:
present_paths=['/home/jose/Documents/GCM/GCM_RCP45_corregido/TMIN/Escenarios_Horizontes_resampleado/present/'+x for x in os.listdir('/home/jose/Documents/GCM/GCM_RCP45_corregido/TMIN/Escenarios_Horizontes_resampleado/present/')]

In [49]:
save_paths=['/home/jose/Documents/GCM/GCM_RCP45_corregido/TMIN/Escenarios_Horizontes_corregidos/'+x.split('.')[0]+'_F_C.nc' for x in os.listdir('/home/jose/Documents/GCM/GCM_RCP45_corregido/TMIN/Escenarios_Horizontes_resampleado/present/')]

In [42]:
save_paths=['/home/jose/Documents/GCM/GCM_RCP45_corregido/TMIN/Escenarios_Horizontes_corregidos/'+x.split('.')[0]+'_P_C.nc' for x in os.listdir('/home/jose/Documents/GCM/GCM_RCP45_corregido/TMIN/Escenarios_Horizontes_resampleado/present/')]

In [50]:
FECHAS=['2017-01-1']


for i in range(len(present_paths)):

    GCM=matrixinfo(present_paths[i],tipo='GCM')  
    FCR=matrixinfo(future_paths[i],'ff')
    #FCR=matrixinfo(present_paths[i],'ff')
    file_saved=save_paths[i]
    nn=FCR.shape[1]
    Resultados=np.zeros((PISCO.shape[0],nn))
    Resultados[:] = np.NAN
    for pixel in tqdm(Elements):
        hist_obs=base.as_numeric(PISCO[pixel,:])
        hist_mod=base.as_numeric(GCM[pixel,:])
        fut_mod=base.as_numeric(FCR[pixel,:])
        try:
            Resultados[pixel,:]=rfunc(hist_obs,hist_mod,fut_mod,FECHAS[0],nn).value
            if pixel%100==0:
                print("Iteracion el el pixel {}".format(pixel))
        except:
            print("An exception occurred")

        #Resultados[Resultados<0]=np.NAN

        #saveNC(fn=file_saved,Resultados=Resultados,fechaI=FECHAS[0])
    saveNC(fn=file_saved,Resultados=Resultados,fechaI=FECHAS[0])     

  0%|          | 17/19791 [00:00<02:00, 163.53it/s]

1008


  1%|          | 115/19791 [00:00<02:05, 156.73it/s]

Iteracion el el pixel 100


  1%|          | 197/19791 [00:01<02:04, 157.56it/s]

...

Iteracion el el pixel 25400


100%|█████████▉| 19717/19791 [02:03<00:00, 160.09it/s]

Iteracion el el pixel 25800


100%|██████████| 19791/19791 [02:04<00:00, 159.21it/s]

Iteracion el el pixel 26200


In [60]:
len(future_paths)

11

In [61]:
len(present_paths)

11